<a href="https://colab.research.google.com/github/nicholasrichers/dissertacao/blob/master/25gb_ram_treinar_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RUNNING San_Francisco


## 1. Install GPU Libs


GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jan 22 07:07:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


Light GBM

In [ ]:
%%capture
!git clone --recursive https://github.com/Microsoft/LightGBM

%cd /content/LightGBM
!mkdir build

!cmake -DUSE_GPU=1
!make -j$(nproc)


In [ ]:
%%capture
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools numpy scipy scikit-learn -U

%cd /content/LightGBM/python-package
!sudo python setup.py install --precompile

%cd /content



---



---



XGboost

In [ ]:
#%%capture
!pip install xgboost==1.2.0

     |████████████████████████████████| 148.9MB 38kB/s 
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90




---



---



Sklearn

In [ ]:
#%%capture
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.23.1

Uninstalling scikit-learn-0.24.1:
  Successfully uninstalled scikit-learn-0.24.1
     |████████████████████████████████| 6.9MB 9.6MB/s 




---



---



## 1.1 Quick Setup dependencies

In [ ]:
#checando versoes para evitar conflitos

import lightgbm as lgbm
print("lgbm version:", lgbm.__version__)

import xgboost as xgb
print("xgb version:", xgb.__version__)

import sklearn
print("sklearn version:", sklearn.__version__)

lgbm version: 3.1.1.99
xgb version: 1.2.0
sklearn version: 0.23.1


Git Clone

In [ ]:
%%capture
!git clone https://github.com/nicholasrichers/dissertacao.git
!pip install scikit-optimize

Git Pull

In [ ]:
%cd /content/dissertacao/
!git pull
%cd /content
#get_libs()

/content/dissertacao
Already up to date.
/content


## 1.2 Import libs

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

def load_library(NAME_LIB, FILE_PATH):
    from importlib.machinery import SourceFileLoader
    FILE_PATH = '/content/dissertacao/' + FILE_PATH
    somemodule = SourceFileLoader(NAME_LIB, FILE_PATH).load_module()
    

load_library('quick_setup', 'src/utils/quick_setup.py')

In [ ]:
#import pandas as pd
from quick_setup import get_libs
get_libs()

In [ ]:
#private modules 
#data
import make_dataset

#models
import train_model
import evaluation
import dsr
import meta_model
import neutralize
fn_strategy = neutralize.fn_strategy_dict

#validation
import group_ts_split
import combinatorial_split
import metrics
import metrics_era
import metrics_description
#import my_validation

#visualization
import visualize

In [ ]:
#main libraries
import numpy as np
import pandas as pd
import scipy
from joblib import dump, load
import matplotlib.pyplot as plt
pd.set_option("display.precision", 4)
%matplotlib inline

#model Libraries
from sklearn import utils
from sklearn import preprocessing

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GroupKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

#skopt
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [ ]:
#private class (tentar treinar com a classe da pasta [no colab] a o inves de colar local)
from sklearn.base import BaseEstimator, RegressorMixin
#class MyXGBRanker(XGBRanker, BaseEstimator, RegressorMixin):

from xgboost import XGBRanker
class MyXGBRanker(XGBRanker, BaseEstimator, RegressorMixin):
    def fit(self, x, y):
        cdf = x.groupby('era').agg(['count'])
        group = cdf[cdf.columns[0]].values
        return super().fit(x[features], y, group=group)

    def predict(self, x):
        return super().predict(x[features])

In [ ]:
results = dict()

## 2. Download the datasets





### training

In [ ]:
%%time
df_training,features,target = make_dataset.get_data(nrows=None,
                                                    low_memory=False, 
                                                    dataset="training", 
                                                    feather=False) #Apenas False colab/aws

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501808 entries, 0 to 501807
Columns: 314 entries, id to target
dtypes: float64(311), int32(1), object(2)
memory usage: 1.2+ GB
None
CPU times: user 16.6 s, sys: 2.07 s, total: 18.6 s
Wall time: 23 s


### tournament

In [ ]:
%%time
df_tournament,features,target = make_dataset.get_data(nrows=None,
                                                    low_memory=False, 
                                                    dataset="tournament", 
                                                    feather=False) #Apenas False colab/aws

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1628412 entries, 0 to 1628411
Columns: 314 entries, id to target
dtypes: float64(311), object(3)
memory usage: 3.8+ GB
None
CPU times: user 53.9 s, sys: 4.57 s, total: 58.5 s
Wall time: 1min 7s


validation

In [ ]:
df_validation = df_tournament[df_tournament.data_type == 'validation']
df_validation['era'] = df_validation.loc[:, 'era'].str[3:].astype('int32')

In [ ]:
df_tournament=None

## 3. Validation Strategy & Metrics

In [ ]:
#Group K-fold
CV = GroupKFold(n_splits = 3)
cv_grp = list(CV.split(X = df_training[features], y = df_training[target],  groups = df_training.era.values))

In [ ]:
#Group TS-fold
CV = group_ts_split.TimeSeriesSplitGroups(n_splits = 5)
ts_grp = list(CV.split(X = df_training[features], y = df_training[target],  groups = df_training.era))

In [ ]:
#Group Slider-fold
CV = group_ts_split.TimeSeriesSplitGroups_Slider(n_splits = 5)
slider_grp = list(CV.split(X = df_training[features], y = df_training[target],  groups = df_training.era))

In [ ]:
#Combinatorial K-fold
#CV = combinatorial_split.CombinatorialPurgedKFold(n_splits=12, n_test_splits=2, samples_info_sets=combinatorial_split.last_ix_series(df_training))
#cpcv_grp = list(CV.split(X = df_training[features], y = df_training['target'],  groups = df_training.era))

In [ ]:
def eras_score(y_true, y_pred):

    #create y_true as df
    y_true = y_true.to_frame(name='target')
    y_true = y_true.join(df_training['era'])

    #create y_pred as df
    preds_df = pd.DataFrame(y_pred, index = y_true.index, columns=['preds'])
    preds_df = preds_df.join(df_training['era'])
    era_scores = pd.Series(index=y_true['era'].unique())

    for era in y_true['era'].unique():
        era_df = y_true[y_true['era'] == era]
        era_preds = preds_df[preds_df['era'] == era]
        era_scores[era] = np.corrcoef(era_df['target'], 
                                      era_preds['preds'].rank(pct=True, method="first"))[0,1]

    
    results = era_scores.to_frame()
    #results.to_csv(file_name_prov, mode='a', header=False, index=True)
    return era_scores


def validation_mean_(y_true, y_pred):
  era_scores = eras_score(y_true, y_pred)
  return np.mean(era_scores)


def annual_sharpe(x):
    return ((np.mean(x) -0.010415154) /np.std(x, ddof=1)) #* np.sqrt(12) 

from scipy.stats import skew, kurtosis, sem, gmean
def adj_sharpe_(y_true, y_pred):
  x = eras_score(y_true, y_pred)
  return annual_sharpe(x) * (1 + ((skew(x) / 6) * annual_sharpe(x)) - \
                             ((kurtosis(x) - 0) / 24) * (annual_sharpe(x) ** 2)) 




scorer = make_scorer(adj_sharpe_)

## 4. FN Strategy (Pre)

### FN Train (metrics)

In [ ]:
import scipy
from sklearn.preprocessing import MinMaxScaler 
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

def _neutralize(df, columns, by, ml_model, proportion=1.0): #['preds'], features,
    scores = df[columns] #preds
    exposures = df[by].values #features
    ml_model[0].fit(exposures, scores.values.reshape(1,-1)[0])
    neutr_preds = pd.DataFrame(ml_model[0].predict(exposures), index=df.index, columns=columns)
    #exposures.dot(np.linalg.pinv(exposures).dot(scores))    

    
    if ml_model[1] != None:
        ml_model[1].fit(exposures, scores.values.reshape(1,-1)[0])
        neutr_preds2 = pd.DataFrame(ml_model[1].predict(exposures), index=df.index, columns=columns)

    else: neutr_preds2 = 0# np.zeros(len(scores))

    scores = scores - ((proportion * neutr_preds) + ((1-proportion) * neutr_preds2))
    #scores = scores - proportion * neutr_preds
    return scores / scores.std()



def _normalize(df):
    X = (df.rank(method="first") - 0.5) / len(df)
    return scipy.stats.norm.ppf(X)


def normalize_and_neutralize(df, columns, by, ml_model, proportion=1.0):
    # Convert the scores to a normal distribution
    df[columns] = _normalize(df[columns])
    #df[columns] = _neutralize(df, columns, by, ml_model, proportion)
    return df[columns]
   


def preds_neutralized(df, columns, by, ml_model, proportion=1.0):

    preds_neutr = df.groupby("era").apply( lambda x: normalize_and_neutralize(x, columns, by, ml_model, proportion))

    preds_neutr = MinMaxScaler().fit_transform(preds_neutr).reshape(1,-1)[0]

    return preds_neutr

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
def adj_sharpe_fn_(y_true, y_pred):

  #create y_true as df
  y_true_ = y_true.to_frame(name='target')
  df_cv = y_true_.join(df_training[['era']+features])
  df_cv['preds'] = y_pred

  y_pred_fn = preds_neutralized(df_cv, ['preds'], features,
                                [LinearRegression(fit_intercept=False), Ridge(alpha=0.5)],
                                0.75)
                             #fn_strategy['nr__medellin']['model'], 
                             #fn_strategy['nr__medellin']['factor'])
  
  x = eras_score(y_true, y_pred_fn)
  return annual_sharpe(x) * (1 + ((skew(x) / 6) * annual_sharpe(x)) - \
                             ((kurtosis(x) - 0) / 24) * (annual_sharpe(x) ** 2)) 

#scorer = make_scorer(adj_sharpe_fn_)


In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_validate, GridSearchCV


#####################################################################
#####################################################################

space  = [
           #step 1 & 6
           Integer(1000, 4000, name='n_estimators', prior='uniform'), 
           Real(5**-3, 10**-1, name='learning_rate', prior='log-uniform'), 

           #step 2
           Integer(3, 10, name='max_depth', prior='uniform'),
           Integer(1, 10, name='min_child_weight', prior='uniform'),

           #step 3
           Real(1e-5, 6, name='gamma', prior='log-uniform'),

           #step 4
           Real(5e-2, .5, name='colsample_bytree', prior='uniform'),
           Real(5e-2, .5, name='subsample', prior='uniform'),
          
           #step 5
           Real(1, 100, name='reg_alpha', prior='log-uniform'),

]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):

    #model.set_params(**params)
    param_grid = {} #hparams in list []
    for key, val in params.items(): param_grid[key] = [val]
    print(params)

    #cria csv prov zerado aqui
    prov_scores =  pd.DataFrame(columns=['scores'])
    prov_scores.to_csv(file_name_prov, header=True, index=0)#df_training.era.unique

    #train model
    grid_search = GridSearchCV(model, param_grid, cv=cv_grp, n_jobs=-1, scoring=scorer)
    grid_search.fit(df_training[features], df_training[target])
    

    #get estimator and results
    model_ = grid_search.best_estimator_
    results = grid_search.cv_results_
    adj_sharpe_val = results['mean_test_score'][0]


    #predict and neutralize results
    df_validation['preds'] = model_.predict(df_validation[features])#.to_list()
    preds_test = neutralize.preds_neutralized(df_validation,
                                              ['preds'], 
                                              features,
                                              fn_strategy['nr__medellin']['model'], 
                                              fn_strategy['nr__medellin']['factor'])
    

    #calculationg eras score
    print("val: ", metrics_era.validation_mean(df_validation[target], preds_test, df_validation))
    eras_score_test = metrics_era.numerai_score(df_validation[target], preds_test, df_validation)
    eras_score_cv = pd.read_csv(file_name_prov, index_col=0)


    ##creating csv
    eras_score_full = pd.concat([eras_score_cv.squeeze(), eras_score_test]).to_frame().T
    eras_score_full['hparam'] = '{}'.format(params)


    eras_score_full.to_csv(file_name, mode='a', header=False, index=False)
    return -adj_sharpe_val

#####################################################################
#####################################################################



print(str(scorer))
model =XGBRegressor(tree_method = 'gpu_hist',
                    #n_estimators=2000,
                    #learning_rate=0.01,
                    #max_depth=5,
                    #colsample_bytree=0.1,
                    n_jobs=-1,
                    gpu_id=0)

file_name = 'era_scores_xgb_slider20.csv'
file_name_prov = 'prov_scores.csv'


#cria csv principal
#cols = cols = group_ts_split.results_df_cols(slider_grp, df_training, df_validation)
#preds_df =  pd.DataFrame(columns=cols)
#preds_df.to_csv(file_name, header=True, index=False)


#res_gp = dummy_minimize(objective, space, n_calls=60, verbose=1, random_state=10)

make_scorer(adj_sharpe_pre_)


### FN Pre (Target)

In [ ]:
def eras_score(y_true, y_pred):
    target = 'target_fn'

    #create y_true as df
    y_true = y_true.to_frame(name=target)
    y_true = y_true.join(df_training['era'])

    #create y_pred as df
    preds_df = pd.DataFrame(y_pred, index = y_true.index, columns=['preds'])
    preds_df = preds_df.join(df_training['era'])
    era_scores = pd.Series(index=y_true['era'].unique())

    for era in y_true['era'].unique():
        era_df = y_true[y_true['era'] == era]
        era_preds = preds_df[preds_df['era'] == era]
        era_scores[era] = np.corrcoef(era_df[target], 
                                      era_preds['preds'].rank(pct=True, method="first"))[0,1]

    
    results = era_scores.to_frame()
    results.to_csv(file_name_prov, mode='a', header=False, index=True)
    return era_scores


def validation_mean_(y_true, y_pred):
  era_scores = eras_score(y_true, y_pred)
  return np.mean(era_scores)


def annual_sharpe(x):
    return ((np.mean(x) -0.010415154) /np.std(x, ddof=1)) #* np.sqrt(12) 

from scipy.stats import skew, kurtosis, sem, gmean
def adj_sharpe_pre_(y_true, y_pred):
  x = eras_score(y_true, y_pred)
  return annual_sharpe(x) * (1 + ((skew(x) / 6) * annual_sharpe(x)) - \
                             ((kurtosis(x) - 0) / 24) * (annual_sharpe(x) ** 2)) 




scorer = make_scorer(adj_sharpe_pre_)

In [ ]:
from sklearn.linear_model import LinearRegression
ml_model = [LinearRegression(fit_intercept=False), None]

df_training['target_fn'] = neutralize.preds_neutralized(df_training, 
                                                        ['target'],
                                                        features,
                                                        ml_model,
                                                        0.8)

In [ ]:
df_training.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target,target_fn
0,n000315175b67977,1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.5,1.0,0.5,0.00,0.50,0.50,0.00,0.0,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.5,0.50,0.75,0.5,...,0.75,0.75,0.50,1.00,1.00,0.50,0.75,0.5,0.25,0.25,0.75,0.50,1.00,0.5,0.75,0.75,0.25,0.50,1.00,0.75,0.50,0.50,1.00,0.25,0.5,0.50,0.50,0.75,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50,0.3874
1,n0014af834a96cdd,1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.5,0.5,0.5,0.50,0.25,0.25,0.50,0.0,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.5,0.25,0.75,0.5,...,0.25,0.25,1.00,1.00,0.50,0.50,0.50,0.0,0.25,1.00,0.50,1.00,1.00,0.5,0.50,0.50,1.00,0.25,0.75,1.00,0.25,0.25,1.00,0.50,0.5,0.50,0.75,0.75,0.75,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25,0.3128
2,n001c93979ac41d4,1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.5,0.5,1.0,0.50,0.50,0.50,0.25,0.0,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.5,0.50,0.75,0.5,...,1.00,1.00,1.00,0.50,1.00,1.00,1.00,0.5,1.00,0.00,1.00,1.00,0.50,1.0,0.75,1.00,0.00,0.50,0.75,0.00,1.00,0.50,0.50,0.75,1.0,0.75,1.00,0.25,0.50,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25,0.3431
3,n0034e4143f22a13,1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.0,0.5,0.5,0.00,1.00,0.00,0.75,0.0,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,0.75,0.75,1.00,0.75,0.75,0.75,1.0,0.75,1.00,0.75,1.00,0.75,1.0,0.00,0.50,0.75,1.00,0.75,1.00,0.75,1.00,1.00,0.00,0.5,0.75,0.75,1.00,0.75,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25,0.3303
4,n00679d1a636062f,1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.0,0.5,0.5,0.25,0.00,0.50,0.00,0.5,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,0.25,0.75,1.00,0.75,0.00,0.00,0.75,0.5,1.00,0.50,0.75,0.25,0.50,0.0,0.50,0.50,0.50,0.75,0.75,0.50,0.75,0.25,0.75,0.50,0.5,0.25,0.25,0.75,0.50,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75,0.5992


### FN Pre (Features)

## 5. Vanilla Models


Estratégia modelo Sao Paulo (colab):

Validacao:
3-GRP fold
random ou skopt?


Modelos
- Linear Regresion
- Orthogonal Matching Pursuit
- Ridge Regression (ou Bayesian ridge)
- Xgboost
- Light GBM
- XGB Ranker

### Slider 5-GRP

#### lineares

linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

lr_param_grid =  {
    'fit_intercept' : [True, False],
    'normalize' : [True, False],

}

results = []
result = evaluation.evaluate_model(df_training[features], df_training[target],
                                   LinearRegression(), 'lr',
                                   lr_param_grid, 
                                   scorer, 
                                   n_iter=1, 
                                   cv_folds=slider_grp, 
                                   pipeline=None)

results.append(result)

==> Starting K-fold cross validation for lr model, 501808 examples
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   24.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   24.3s finished


==> Elapsed seconds: 35.859
Best lr model: LinearRegression(fit_intercept=False, normalize=True)
Best lr score (val): 0.9596




---



---



Ridge 

In [ ]:
from sklearn.linear_model import Ridge

ridge_param_grid =  {
    'fit_intercept' : [True, False],
    'normalize' : [True, False],
    'alpha' : Real(10**-1, 10**1, "log-uniform")

}

print(grp_type)
result = evaluation.evaluate_model_skopt(df_training[features], df_training[target],
                                          Ridge(), 'ridge',
                                          ridge_param_grid, 
                                          scorer, 
                                          n_iter=10, 
                                          cv_folds=slider_grp, 
                                          pipeline=None)

results.append(result)



---



---



OMP

In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit

omp_param_grid =  {
    'n_nonzero_coefs' : Integer(1, len(features), 'uniform')

}

print(grp_type)
result = evaluation.evaluate_model_skopt(df_training[features], df_training[target],
                                   OrthogonalMatchingPursuit(), 'omp',
                                   omp_param_grid, 
                                   scorer, 
                                   n_iter=10, 
                                   cv_folds=slider_grp, 
                                   pipeline=None)

results.append(result)



---



---



#### GBM

##### xgboost vanilla

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_validate, GridSearchCV


#####################################################################
#####################################################################

space  = [
           #step 1 & 6
           Integer(1000, 4000, name='n_estimators', prior='uniform'), 
           Real(5**-3, 10**-1, name='learning_rate', prior='log-uniform'), 

           #step 2
           Integer(3, 10, name='max_depth', prior='uniform'),
           Integer(1, 10, name='min_child_weight', prior='uniform'),

           #step 3
           Real(1e-5, 6, name='gamma', prior='log-uniform'),

           #step 4
           Real(5e-2, .5, name='colsample_bytree', prior='uniform'),
           Real(5e-2, .5, name='subsample', prior='uniform'),
          
           #step 5
           Real(1, 100, name='reg_alpha', prior='log-uniform'),

]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):

    #model.set_params(**params)
    param_grid = {} #hparams in list []
    for key, val in params.items(): param_grid[key] = [val]
    print(params)

    #cria csv prov zerado aqui
    prov_scores =  pd.DataFrame(columns=['scores'])
    prov_scores.to_csv(file_name_prov, header=True, index=0)#df_training.era.unique

    #train model
    #print(model)
    grid_search = GridSearchCV(model, param_grid, cv=slider_grp, n_jobs=-1, scoring=scorer)
    grid_search.fit(df_training[features], df_training[target])
    

    #get estimator and results
    model_ = grid_search.best_estimator_
    results = grid_search.cv_results_
    adj_sharpe_val = results['mean_test_score'][0]


    #predict and neutralize results
    preds_test = model_.predict(df_validation[features])#.to_list()
    #df_validation['preds'] = model_.predict(df_validation[features])#.to_list()
    #preds_test = neutralize.preds_neutralized(df_validation,
                                              #['preds'], 
                                              #features,
                                              #fn_strategy['nr__medellin']['model'], 
                                              #fn_strategy['nr__medellin']['factor'])
    

    #calculationg eras score
    print("val: ", metrics_era.validation_mean(df_validation['target'], preds_test, df_validation))
    eras_score_test = metrics_era.numerai_score(df_validation['target'], preds_test, df_validation)
    eras_score_cv = pd.read_csv(file_name_prov, index_col=0)


    ##creating csv
    eras_score_full = pd.concat([eras_score_cv.squeeze(), eras_score_test]).to_frame().T
    eras_score_full['hparam'] = '{}'.format(params)


    eras_score_full.to_csv(file_name, mode='a', header=False, index=False)
    return -adj_sharpe_val

#####################################################################
#####################################################################


target = 'target_fn'
print(str(scorer))
print(target)
model =XGBRegressor(tree_method = 'gpu_hist',
                    #n_estimators=2000,
                    #learning_rate=0.01,
                    #max_depth=5,
                    #colsample_bytree=0.1,
                    n_jobs=-1,
                    gpu_id=0)

file_name = 'era_scores_xgb_slider20.csv'
file_name_prov = 'prov_scores.csv'


#cria csv principal
cols = cols = group_ts_split.results_df_cols(slider_grp, df_training, df_validation)
preds_df =  pd.DataFrame(columns=cols)
preds_df.to_csv(file_name, header=True, index=False)


res_gp = dummy_minimize(objective, space, n_calls=30, verbose=1, random_state=300)

make_scorer(adj_sharpe_pre_)
target_fn
Iteration No: 1 started. Evaluating function at random point.
{'n_estimators': 3529, 'learning_rate': 0.013980786598051327, 'max_depth': 5, 'min_child_weight': 10, 'gamma': 0.00047879101992313357, 'colsample_bytree': 0.15848506737625978, 'subsample': 0.4019208948320882, 'reg_alpha': 41.89756003444102}
val:  0.01971690104993401
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 107.6041
Function value obtained: 0.2075
Current minimum: 0.2075
Iteration No: 2 started. Evaluating function at random point.
{'n_estimators': 2394, 'learning_rate': 0.058323942446671245, 'max_depth': 6, 'min_child_weight': 4, 'gamma': 0.043148547714635645, 'colsample_bytree': 0.343364394392695, 'subsample': 0.43636599675502347, 'reg_alpha': 14.914389957208082}
val:  0.01544465282032878
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 124.3490
Function value obtained: -0.1191
Current minimum: -0.1191
Iteration No: 3 started. Evaluating fu

In [ ]:
from google.colab import files
files.download("/content/era_scores_xgb_slider20.csv")

In [ ]:
from xgboost import XGBRegressor
#example scripts
#XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1)



b={'n_estimators': 2851, 
 'learning_rate': 0.012559878784975307, 
 'max_depth': 4, 
 'min_child_weight': 7, 
 'gamma': 0.17985836491648036, 
 'colsample_bytree': 0.2784204086247318, 
 'subsample': 0.33994697676863767, 
 'reg_alpha': 2.6691509215708877}


xgb_param_grid = {
        #step 1 & 6
        'n_estimators' : [2851],#Integer(1000, 5000, 'uniform'),
        'learning_rate' : [0.012559878784975307],#Real(10**-3, 10**-1, "log-uniform"),


        #step 2
        'max_depth' : [4],#Integer(3, 10, 'uniform'),
        'min_child_weight':[7],# Integer(1, 10, 'uniform'),

        #step 3
        'gamma' : [0.17985836491648036],#Real(1e-5, 6, 'log-uniform'),


        #step 4
        'subsample' : [0.33994697676863767],#Real(1e-1, 1, 'uniform'),
        'colsample_bytree':[0.2784204086247318], #Real(1e-1, .5, 'uniform'), 

        #step 5
        'reg_alpha' : [2.6691509215708877],#Real(1, 100, 'log-uniform'),


        #utils
        'objective': ["reg:squarederror"],
        #'early_stopping_rounds' : [30],
        'tree_method' : ['gpu_hist'],#['gpu_hist'], 
        'gpu_id' : [0],
        
}


gpu_params = { 'predictor':'cpu_predictor', 'tree_method':'hist'}
result = evaluation.evaluate_model(df_training[features], df_training[target],
                                          XGBRegressor(), 'xgb_slider20',
                                          xgb_param_grid, 
                                          scorer, 
                                          n_jobs=-1, 
                                          n_iter=1, 
                                          cv_folds=slider_grp, 
                                          pipeline=None)


result[0].model.get_booster().set_param(gpu_params)
results['xgb_slider20'] = result

==> Starting K-fold cross validation for xgb_slider20 model, 501808 examples
[1.69657795 0.84173346 0.65021149 0.44486033 0.41062111]
==> Elapsed seconds: 238.180
Best xgb_slider20 model: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.22923619312392746,
             gamma=0.009652923007014539, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.015105554114971671,
             max_delta_step=0, max_depth=9, min_child_weight=1, missing=nan,
             monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
             n_estimators=4646, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=24.748680026879075, reg_lambda=1, scale_pos_weight=1,
             subsample=0.32960762834422813, tree_method='gpu_hist',
             valid

##### lgbm vanilla

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_validate, GridSearchCV


#####################################################################
#####################################################################

space  = [
           #step 1&5
           Integer(1000, 4000, name='n_estimators', prior='uniform'), #1000,4000
           Real(5**-3, 10**-1,  name='learning_rate', prior='log-uniform'),

           #step 2
           Integer(50, 250, name='num_leaves', prior='uniform'),
           Integer(100, 2000, name='min_data_in_leaf', prior='uniform'),
           Integer(3, 30, name='max_depth', prior='uniform'),

           #step 3
           Real(5e-2, 0.5, name='colsample_bytree', prior='uniform'),
           Real(5e-2, 0.5, name='subsample', prior='uniform'),

           #step 4
           Real(10**-2, 1.0, name='reg_lambda', prior='log-uniform'),
]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):

    #model.set_params(**params)
    param_grid = {} #hparams in list []
    for key, val in params.items(): param_grid[key] = [val]
    print(params)

    #cria csv prov zerado aqui
    prov_scores =  pd.DataFrame(columns=['scores'])
    prov_scores.to_csv(file_name_prov, header=True, index=0)#df_training.era.unique

    #train model
    print(model)
    grid_search = GridSearchCV(model, param_grid, cv=slider_grp, n_jobs=-1, scoring=scorer)
    grid_search.fit(df_training[features], df_training[target])
    

    #get estimator and results
    model_ = grid_search.best_estimator_
    results = grid_search.cv_results_
    adj_sharpe_val = results['mean_test_score'][0]


    #predict and neutralize results
    preds_test = model_.predict(df_validation[features])#.to_list()
    #df_validation['preds'] = model_.predict(df_validation[features])#.to_list()
    #preds_test = neutralize.preds_neutralized(df_validation,
                                              #['preds'], 
                                              #features,
                                              #fn_strategy['nr__medellin']['model'], 
                                              #fn_strategy['nr__medellin']['factor'])
    

    #calculationg eras score
    print("val: ", metrics_era.validation_mean(df_validation['target'], preds_test, df_validation))
    eras_score_test = metrics_era.numerai_score(df_validation['target'], preds_test, df_validation)
    eras_score_cv = pd.read_csv(file_name_prov, index_col=0)


    ##creating csv
    eras_score_full = pd.concat([eras_score_cv.squeeze(), eras_score_test]).to_frame().T
    eras_score_full['hparam'] = '{}'.format(params)


    eras_score_full.to_csv(file_name, mode='a', header=False, index=False)
    return -adj_sharpe_val

#####################################################################
#####################################################################


target = 'target_fn'
print(str(scorer))
print(target)
model =LGBMRegressor(device_type = 'gpu',
                    #n_estimators=2000,
                    #learning_rate=0.01,
                    #max_depth=5,
                    #colsample_bytree=0.1,
                    boosting_type= 'goss',
                    n_jobs=-1)

file_name = 'era_scores_lgbm_slider20.csv'
file_name_prov = 'prov_scores.csv'


#cria csv principal
cols =  group_ts_split.results_df_cols(slider_grp, df_training, df_validation)
preds_df =  pd.DataFrame(columns=cols)
preds_df.to_csv(file_name, header=True, index=False)


res_gp = dummy_minimize(objective, space, n_calls=10, verbose=1, random_state=200)

In [ ]:
from google.colab import files
files.download("/content/era_scores_lgbm_slider20.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from lightgbm import LGBMRegressor
#LGBMRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, num_leaves=32)



lgbm_best={'n_estimators': 2258, 'learning_rate': 0.008203629273160581, 
 'num_leaves': 92, 'min_data_in_leaf': 1395, 'max_depth': 19, 
 'colsample_bytree': 0.14430607619531413, 
 'subsample': 0.17193342288927632, 
 'reg_lambda': 0.13942748601876273,
 'boosting_type': 'goss',
  'device_type' : 'gpu'
 }


lgbm_param_grid = {
    
    #step 1 & 6
    'n_estimators' : [2258],#Integer(500, 4000, 'uniform'), #[623]
    'learning_rate' :[0.008203629273160581],# Real(10**-3, 10**-1, "log-uniform"),  #[0.1]


    #step 2
    'num_leaves': [92], #Integer(50, 250, "uniform"), #
    'min_data_in_leaf' : [1395],# Integer(100, 2000, 'uniform'), #
    'max_depth' : [19], #Integer(3, 30, 'uniform'), #
    

    #step 3
    #'bagging_fraction' : [0.3195476883574932],#Real(1e-1, 1, 'uniform'), #
    #'bagging_temperature': [0.5776416670125987],# Real(1e-1, 1.0, 'uniform'), #
    
    #step 4
    'colsample_bytree': [0.14430607619531413], #Real(1e-1, .5, 'uniform'), #
    'subsample': [0.17193342288927632], #Real(1e-1, 1.0, 'uniform'), #

    #step 5
    'reg_lambda': [0.13942748601876273], #Real(10**-2, 1, "log-uniform"), #


    #utils
    'boosting_type': ['goss'],
    'device_type' : ['gpu']

}



#gpu_params = {'device_type':'cpu'}
result = evaluation.evaluate_model(df_training[features], df_training[target],
                                          LGBMRegressor(), 'lgbm_slider20',
                                          lgbm_param_grid, 
                                          scorer, 
                                          n_iter=1, 
                                          cv_folds=slider_grp, 
                                          pipeline=None)


#result[0].model.set_params(**gpu_params)
results['lgbm_slider20'] = result

##### XGB Ranker

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRanker
from sklearn.model_selection import cross_validate


#####################################################################
#####################################################################

space  = [
           #step 1 & 6
           Integer(2000, 5000, name='n_estimators', prior='uniform'), 
           Real(5**-3, 10**-1, name='learning_rate', prior='log-uniform'), 

           #step 2
           Integer(3, 10, name='max_depth', prior='uniform'),
           Integer(1, 10, name='min_child_weight', prior='uniform'),

           #step 3
           Real(1e-5, 6, name='gamma', prior='log-uniform'),

           #step 4
           Real(5e-2, .5, name='colsample_bytree', prior='uniform'),
           Real(5e-2, .5, name='subsample', prior='uniform'),
          
           #step 5
           Real(1, 100, name='reg_alpha', prior='log-uniform'),

]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    #print(params)

    #cria csv prov zerado aqui
    prov_scores =  pd.DataFrame(columns=['scores'])
    prov_scores.to_csv(file_name_prov, header=True, index=0)#df_training.era.unique


    res_dict = cross_validate(model, 
                               df_training[features], 
                               df_training[target], 
                               groups=df_training.era,
                               return_estimator=True,
                               #return_train_score=True,
                               scoring = scorer, 
                               n_jobs=-1,
                               cv=slider_grp)
    

    adj_sharpe_val = res_dict['test_score'].mean()
    preds_test = res_dict['estimator'][-1].predict(df_validation[features]).tolist()


    eras_score_test = metrics_era.numerai_score(df_validation[target], preds_test, df_validation)
    eras_score_cv = pd.read_csv(file_name_prov, index_col=0)
    

    eras_score_full = pd.concat([eras_score_cv.squeeze(), eras_score_test]).to_frame().T
    eras_score_full['hparam'] = '{}'.format(params)


    eras_score_full.to_csv(file_name, mode='a', header=False, index=False)
    return -adj_sharpe_val

#####################################################################
#####################################################################

class MyXGBRanker(XGBRanker, BaseEstimator, RegressorMixin):
    def fit(self, x, y):
        cdf = x.groupby('era').agg(['count'])
        group = cdf[cdf.columns[0]].values
        return super().fit(x[features], y, group=group)

    def predict(self, x):
        return super().predict(x[features])


print(str(scorer))
model =MyXGBRanker(tree_method = 'gpu_hist',
                    #n_estimators=2000,
                    #learning_rate=0.01,
                    #max_depth=5,
                    #colsample_bytree=0.1,
                    n_jobs=-1,
                    gpu_id=0)

file_name = 'era_scores_xgb_ranker_slider5.csv'
file_name_prov = 'prov_scores.csv'


#cria csv principal
cols = cols = group_ts_split.results_df_cols(slider_grp, df_training, df_validation)
preds_df =  pd.DataFrame(columns=cols)
preds_df.to_csv(file_name, header=True, index=False)


res_gp = dummy_minimize(objective, space, n_calls=200, verbose=1, random_state=54)

In [ ]:
from xgboost import XGBRanker
from sklearn.base import BaseEstimator, RegressorMixin

class MyXGBRanker(XGBRanker, BaseEstimator, RegressorMixin):
    def fit(self, x, y):
        cdf = x.groupby('era').agg(['count'])
        group = cdf[cdf.columns[0]].values
        return super().fit(x[features], y, group=group)

    def predict(self, x):
        return super().predict(x[features])


xgb_param_grid = {
        #step 1 & 6
        'n_estimators' : [4547], #Integer(3000, 6000, 'uniform'),
        'learning_rate' : [0.01], #Real(10**-3, 10**-1, "log-uniform"), 


        #step 2
        'max_depth' : [3], #Integer(3, 10, 'uniform'),
        'min_child_weight':[9], #Integer(8, 16, 'uniform'), 

        #step 3
        'gamma' : [0.00003253],# Real(1e-5, 6, 'log-uniform'),


        #step 4
        'subsample' : [1],# Real(1e-1, 1, 'uniform'),
        'colsample_bytree':[0.1553875313199311],# Real(1e-1, .5, 'uniform'), 

        #step 5
        'reg_alpha' : [50.20283749898673], #Real(1, 100, 'log-uniform'),

        
        #utils
        #'early_stopping_rounds' : [30],
        'tree_method' : ['gpu_hist'],#['gpu_hist'], 
        'gpu_id' : [0],

        
}


#gpu_params = {'updater':'', 'predictor':'cpu_predictor', 'tree_method':'hist'}
result = evaluation.evaluate_model_skopt(df_training[['era']+features], df_training[target],
                                          MyXGBRanker(), 'xgb_ranker',
                                          xgb_param_grid, 
                                          scorer, 
                                          n_jobs=-1, 
                                          n_iter=1, 
                                          cv_folds=cv_grp, 
                                          pipeline=None)

#result[0].model.get_booster().set_param(gpu_params)
results['xgb_ranker'] = result

#step 1: 0.0352
#step 2: 0.0460
#step 3: 0.0460
#step 4: 0.0457
#step 5: 0.0465
#step 6: 0.0480

#### Salvando os modelos (vanilla)

In [ ]:
model_name = [
              #"lr"
              #,"ridge"
              #,"omp"
              #"xgb_slider20"
              "lgbm_slider20"
              #"xgb_ranker"             
]

In [ ]:
from joblib import dump, load

#Maior media, senao sera ultimo feito, 
#results = sorted(results, key=lambda x: x[2], reverse=True)


for name in model_name:
  #model = list(filter(lambda x: x[1] == name, results))[-1][0]
  dump(results[name][0], name + '-cv.pkl')

### Expanding 5-GRP

#### lineares

linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

lr_param_grid =  {
    'fit_intercept' : [True, False],
    'normalize' : [True, False],

}


result = evaluation.evaluate_model(df_training[features], df_training[target],
                                   LinearRegression(), 'lr',
                                   lr_param_grid, 
                                   scorer, 
                                   n_iter=1, 
                                   cv_folds=ts_grp, 
                                   pipeline=None)

results.append(result)

==> Starting K-fold cross validation for lr model, 501808 examples
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   24.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   24.3s finished


==> Elapsed seconds: 35.859
Best lr model: LinearRegression(fit_intercept=False, normalize=True)
Best lr score (val): 0.9596




---



---



Ridge 

In [ ]:
from sklearn.linear_model import Ridge

ridge_param_grid =  {
    'fit_intercept' : [True, False],
    'normalize' : [True, False],
    'alpha' : Real(10**-1, 10**1, "log-uniform")

}


result = evaluation.evaluate_model(df_training[features], df_training[target],
                                          Ridge(), 'ridge',
                                          ridge_param_grid, 
                                          scorer, 
                                          n_iter=1, 
                                          cv_folds=ts_grp, 
                                          pipeline=None)

results['ridge'] = result

==> Starting K-fold cross validation for ridge model, 501808 examples
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    4.8s remaining:    7.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.3s finished


==> Elapsed seconds: 9.304
Best ridge model: Ridge(alpha=40)
Best ridge score (val): 0.7682




---



---



OMP

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import cross_validate


#####################################################################
#####################################################################

space  = [    
           Categorical([True, False], name='fit_intercept'),
           Categorical([True, False], name='normalize'),
           Integer(10, len(features), name='n_nonzero_coefs', prior='uniform')
]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    #print(params)

    #cria csv prov zerado aqui
    prov_scores =  pd.DataFrame(columns=['scores'])
    prov_scores.to_csv(file_name_prov, header=True, index=0)#df_training.era.unique


    res_dict = cross_validate(model, 
                               df_training[features], 
                               df_training[target], 
                               groups=df_training.era,
                               return_estimator=True,
                               #return_train_score=True,
                               scoring = scorer, 
                               n_jobs=-1,
                               cv=ts_grp)
    

    adj_sharpe_val = res_dict['test_score'].mean()
    preds_test = res_dict['estimator'][-1].predict(df_validation[features]).tolist()


    eras_score_test = metrics_era.numerai_score(df_validation[target], preds_test, df_validation)
    eras_score_cv = pd.read_csv(file_name_prov, index_col=0)
    

    eras_score_full = pd.concat([eras_score_cv.squeeze(), eras_score_test]).to_frame().T
    eras_score_full['hparam'] = '{}'.format(params)


    eras_score_full.to_csv(file_name, mode='a', header=False, index=False)
    return -adj_sharpe_val

#####################################################################
#####################################################################


print(str(scorer))
model =OrthogonalMatchingPursuit()

file_name = 'era_scores_omp_exp20.csv'
file_name_prov = 'prov_scores.csv'


#cria csv principal
cols = cols = group_ts_split.results_df_cols(ts_grp, df_training, df_validation)
preds_df =  pd.DataFrame(columns=cols)
preds_df.to_csv(file_name, header=True, index=False)


res_gp = dummy_minimize(objective, space, n_calls=200, verbose=1, random_state=34)

In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit

omp_param_grid =  {
    'fit_intercept' : [True],
    'normalize' : [False],
    'n_nonzero_coefs' : [92],#Integer(1, len(features), 'uniform')

}


result = evaluation.evaluate_model_skopt(df_training[features], df_training[target],
                                   OrthogonalMatchingPursuit(), 'omp_exp20',
                                   omp_param_grid, 
                                   scorer, 
                                   n_iter=1, 
                                   cv_folds=ts_grp, 
                                   pipeline=None)


results['omp_exp20'] = result

==> Starting K-fold cross validation for omp_exp20 model, 501808 examples
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.5s finished


==> Elapsed seconds: 11.128
Best omp_exp20 model: OrthogonalMatchingPursuit(n_nonzero_coefs=92, normalize=False)
Best omp_exp20 score (val): 0.7894




---



---



#### GBM

##### xgboost vanilla

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_validate


#####################################################################
#####################################################################

space  = [
           #step 1 & 6
           Integer(2000, 5000, name='n_estimators', prior='uniform'), 
           Real(5**-3, 10**-1, name='learning_rate', prior='log-uniform'), 

           #step 2
           Integer(3, 10, name='max_depth', prior='uniform'),
           Integer(1, 10, name='min_child_weight', prior='uniform'),

           #step 3
           Real(1e-5, 6, name='gamma', prior='log-uniform'),

           #step 4
           Real(5e-2, .5, name='colsample_bytree', prior='uniform'),
           Real(5e-2, .5, name='subsample', prior='uniform'),
          
           #step 5
           Real(1, 100, name='reg_alpha', prior='log-uniform'),

]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    #print(params)

    #cria csv prov zerado aqui
    prov_scores =  pd.DataFrame(columns=['scores'])
    prov_scores.to_csv(file_name_prov, header=True, index=0)#df_training.era.unique


    res_dict = cross_validate(model, 
                               df_training[features], 
                               df_training[target], 
                               groups=df_training.era,
                               return_estimator=True,
                               #return_train_score=True,
                               scoring = scorer, 
                               n_jobs=-1,
                               cv=ts_grp)
    

    adj_sharpe_val = res_dict['test_score'].mean()
    preds_test = res_dict['estimator'][-1].predict(df_validation[features]).tolist()


    eras_score_test = metrics_era.numerai_score(df_validation[target], preds_test, df_validation)
    eras_score_cv = pd.read_csv(file_name_prov, index_col=0)
    

    eras_score_full = pd.concat([eras_score_cv.squeeze(), eras_score_test]).to_frame().T
    eras_score_full['hparam'] = '{}'.format(params)


    eras_score_full.to_csv(file_name, mode='a', header=False, index=False)
    return -adj_sharpe_val

#####################################################################
#####################################################################

print(str(scorer))
model =XGBRegressor(tree_method = 'gpu_hist',
                    #n_estimators=2000,
                    #learning_rate=0.01,
                    #max_depth=5,
                    #colsample_bytree=0.1,
                    n_jobs=-1,
                    gpu_id=0)

file_name = 'era_scores_xgb_exp20.csv'
file_name_prov = 'prov_scores.csv'


#cria csv principal
cols = group_ts_split.results_df_cols(ts_grp, df_training, df_validation)
preds_df =  pd.DataFrame(columns=cols)
preds_df.to_csv(file_name, header=True, index=False)


res_gp = dummy_minimize(objective, space, n_calls=200, verbose=1, random_state=154)

In [ ]:
from google.colab import files
files.download("/content/era_scores_xgb_exp20.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from xgboost import XGBRegressor
#example scripts
#XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1)



b={'n_estimators': 2851, 
 'learning_rate': 0.012559878784975307, 
 'max_depth': 4, 
 'min_child_weight': 7, 
 'gamma': 0.17985836491648036, 
 'colsample_bytree': 0.2784204086247318, 
 'subsample': 0.33994697676863767, 
 'reg_alpha': 2.6691509215708877}


xgb_param_grid = {
        #step 1 & 6
        'n_estimators' : [2851],#Integer(1000, 5000, 'uniform'),
        'learning_rate' : [0.012559878784975307],#Real(10**-3, 10**-1, "log-uniform"),


        #step 2
        'max_depth' : [4],#Integer(3, 10, 'uniform'),
        'min_child_weight':[7],# Integer(1, 10, 'uniform'),

        #step 3
        'gamma' : [0.17985836491648036],#Real(1e-5, 6, 'log-uniform'),


        #step 4
        'subsample' : [0.33994697676863767],#Real(1e-1, 1, 'uniform'),
        'colsample_bytree':[0.2784204086247318], #Real(1e-1, .5, 'uniform'), 

        #step 5
        'reg_alpha' : [2.6691509215708877],#Real(1, 100, 'log-uniform'),


        #utils
        'objective': ["reg:squarederror"],
        #'early_stopping_rounds' : [30],
        'tree_method' : ['gpu_hist'],#['gpu_hist'], 
        'gpu_id' : [0],
        
}


gpu_params = { 'predictor':'cpu_predictor', 'tree_method':'hist'}
result = evaluation.evaluate_model(df_training[features], df_training[target],
                                          XGBRegressor(), 'xgb_slider20',
                                          xgb_param_grid, 
                                          scorer, 
                                          n_jobs=-1, 
                                          n_iter=1, 
                                          cv_folds=ts_grp, 
                                          pipeline=None)


result[0].model.get_booster().set_param(gpu_params)
results['xgb_slider20'] = result

==> Starting K-fold cross validation for xgb_slider20 model, 501808 examples
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.2min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.7min finished


==> Elapsed seconds: 132.407
Best xgb_slider20 model: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.2784204086247318,
             gamma=0.17985836491648036, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.012559878784975307,
             max_delta_step=0, max_depth=4, min_child_weight=7, missing=nan,
             monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
             n_estimators=2851, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=2.6691509215708877, reg_lambda=1, scale_pos_weight=1,
             subsample=0.33994697676863767, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)
Best xgb_slider20 score (val): 0.7625


##### lgbm vanilla

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_validate


#####################################################################
#####################################################################

space  = [
           #step 1&5
           Integer(1000, 4000, name='n_estimators', prior='uniform'),
           Real(5**-3, 10**-1,  name='learning_rate', prior='log-uniform'),

           #step 2
           Integer(50, 250, name='num_leaves', prior='uniform'),
           Integer(100, 2000, name='min_data_in_leaf', prior='uniform'),
           Integer(3, 30, name='max_depth', prior='uniform'),

           #step 3
           Real(5e-2, 0.5, name='colsample_bytree', prior='uniform'),
           Real(5e-2, 0.5, name='subsample', prior='uniform'),

           #step 4
           Real(10**-2, 1.0, name='reg_lambda', prior='log-uniform'),
]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    #print(params)

    #cria csv prov zerado aqui
    prov_scores =  pd.DataFrame(columns=['scores'])
    prov_scores.to_csv(file_name_prov, header=True, index=0)#df_training.era.unique


    res_dict = cross_validate(model, 
                               df_training[features], 
                               df_training[target], 
                               groups=df_training.era,
                               return_estimator=True,
                               #return_train_score=True,
                               scoring = scorer, 
                               n_jobs=-1,
                               cv=ts_grp)
    

    adj_sharpe_val = res_dict['test_score'].mean()
    preds_test = res_dict['estimator'][-1].predict(df_validation[features]).tolist()


    eras_score_test = metrics_era.numerai_score(df_validation[target], preds_test, df_validation)
    eras_score_cv = pd.read_csv(file_name_prov, index_col=0)
    

    eras_score_full = pd.concat([eras_score_cv.squeeze(), eras_score_test]).to_frame().T
    eras_score_full['hparam'] = '{}'.format(params)


    eras_score_full.to_csv(file_name, mode='a', header=False, index=False)
    return -adj_sharpe_val

#####################################################################
#####################################################################

print(str(scorer))
model =LGBMRegressor(device_type = 'gpu',
                    #n_estimators=2000,
                    #learning_rate=0.01,
                    #max_depth=5,
                    #colsample_bytree=0.1,
                    n_jobs=-1)

file_name = 'era_scores_lgbm_exp5.csv'
file_name_prov = 'prov_scores.csv'


#cria csv principal
cols = group_ts_split.results_df_cols(slider_grp, df_training, df_validation)
preds_df = pd.DataFrame(columns=cols)
preds_df.to_csv(file_name, header=True, index=False)


res_gp = dummy_minimize(objective, space, n_calls=200, verbose=1, random_state=54)

In [ ]:
from lightgbm import LGBMRegressor
#LGBMRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, num_leaves=32)


b={'n_estimators': 2012, 
 'learning_rate': 0.008053288987896146, 
 'num_leaves': 104, 
 'min_data_in_leaf': 1877, 
 'max_depth': 19, 
 'colsample_bytree': 0.2838163562602001, 
 'subsample': 0.2265121308916066, 
 'reg_lambda': 0.4781007325965136}

lgbm_param_grid = {
    

    #step 1 & 6
    'n_estimators' : [2012],#Integer(500, 4000, 'uniform'), #[623]
    'learning_rate' :[0.008053288987896146],# Real(10**-3, 10**-1, "log-uniform"),  #[0.1]


    #step 2
    'num_leaves': [104], #Integer(50, 250, "uniform"), #
    'min_data_in_leaf' : [1877],# Integer(100, 2000, 'uniform'), #
    'max_depth' : [19], #Integer(3, 30, 'uniform'), #
    

    #step 3
    #'bagging_fraction' : [0.3195476883574932],#Real(1e-1, 1, 'uniform'), #
    #'bagging_temperature': [0.5776416670125987],# Real(1e-1, 1.0, 'uniform'), #
    
    #step 4
    'colsample_bytree': [0.2838163562602001], #Real(1e-1, .5, 'uniform'), #
    'subsample': [0.2265121308916066], #Real(1e-1, 1.0, 'uniform'), #

    #step 5
    'reg_lambda': [0.4781007325965136], #Real(10**-2, 1, "log-uniform"), #


    #utils
    'boosting_type': ['goss'],
    'device_type' : ['gpu']

}



gpu_params = {'device_type':'cpu'}
result = evaluation.evaluate_model(df_training[features], df_training[target],
                                          LGBMRegressor(), 'lgbm_cv40',
                                          lgbm_param_grid, 
                                          scorer, 
                                          n_iter=1, 
                                          cv_folds=cv_grp, 
                                          pipeline=None)


result[0].model.set_params(**gpu_params)
results['lgbm_cv40'] = result

==> Starting K-fold cross validation for lgbm_exp20 model, 501808 examples
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: ignored

##### XGB Ranker

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRanker
from sklearn.model_selection import cross_validate


#####################################################################
#####################################################################

space  = [
           #step 1 & 6
           Integer(2000, 5000, name='n_estimators', prior='uniform'), 
           Real(5**-3, 10**-1, name='learning_rate', prior='log-uniform'), 

           #step 2
           Integer(3, 10, name='max_depth', prior='uniform'),
           Integer(1, 10, name='min_child_weight', prior='uniform'),

           #step 3
           Real(1e-5, 6, name='gamma', prior='log-uniform'),

           #step 4
           Real(5e-2, .5, name='colsample_bytree', prior='uniform'),
           Real(5e-2, .5, name='subsample', prior='uniform'),
          
           #step 5
           Real(1, 100, name='reg_alpha', prior='log-uniform'),

]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    #print(params)

    #cria csv prov zerado aqui
    prov_scores =  pd.DataFrame(columns=['scores'])
    prov_scores.to_csv(file_name_prov, header=True, index=0)#df_training.era.unique


    res_dict = cross_validate(model, 
                               df_training[features], 
                               df_training[target], 
                               groups=df_training.era,
                               return_estimator=True,
                               #return_train_score=True,
                               scoring = scorer, 
                               n_jobs=-1,
                               cv=ts_grp)
    

    adj_sharpe_val = res_dict['test_score'].mean()
    preds_test = res_dict['estimator'][-1].predict(df_validation[features]).tolist()


    eras_score_test = metrics_era.numerai_score(df_validation[target], preds_test, df_validation)
    eras_score_cv = pd.read_csv(file_name_prov, index_col=0)
    

    eras_score_full = pd.concat([eras_score_cv.squeeze(), eras_score_test]).to_frame().T
    eras_score_full['hparam'] = '{}'.format(params)


    eras_score_full.to_csv(file_name, mode='a', header=False, index=False)
    return -adj_sharpe_val

#####################################################################
#####################################################################


print(str(scorer))
model =XGBRegressor(tree_method = 'gpu_hist',
                    objective='rank:pairwise',
                    #n_estimators=2000,
                    #learning_rate=0.01,
                    #max_depth=5,
                    #colsample_bytree=0.1,
                    n_jobs=-1,
                    gpu_id=0)

file_name = 'era_scores_xgb_ranker_exp20.csv'
file_name_prov = 'prov_scores.csv'


#cria csv principal
cols = cols = group_ts_split.results_df_cols(ts_grp, df_training, df_validation)
preds_df =  pd.DataFrame(columns=cols)
preds_df.to_csv(file_name, header=True, index=False)


res_gp = dummy_minimize(objective, space, n_calls=200, verbose=1, random_state=54)

In [ ]:
from google.colab import files
files.download("/content/era_scores_xgb_ranker_exp20.csv")

In [ ]:
from xgboost import XGBRanker
from sklearn.base import BaseEstimator, RegressorMixin

class MyXGBRanker(XGBRanker, BaseEstimator, RegressorMixin):
    def fit(self, x, y):
        cdf = x.groupby('era').agg(['count'])
        group = cdf[cdf.columns[0]].values
        return super().fit(x[features], y, group=group)

    def predict(self, x):
        return super().predict(x[features])


xgb_param_grid = {
        #step 1 & 6
        'n_estimators' : [4547], #Integer(3000, 6000, 'uniform'),
        'learning_rate' : [0.01], #Real(10**-3, 10**-1, "log-uniform"), 


        #step 2
        'max_depth' : [3], #Integer(3, 10, 'uniform'),
        'min_child_weight':[9], #Integer(8, 16, 'uniform'), 

        #step 3
        'gamma' : [0.00003253],# Real(1e-5, 6, 'log-uniform'),


        #step 4
        'subsample' : [1],# Real(1e-1, 1, 'uniform'),
        'colsample_bytree':[0.1553875313199311],# Real(1e-1, .5, 'uniform'), 

        #step 5
        'reg_alpha' : [50.20283749898673], #Real(1, 100, 'log-uniform'),

        
        #utils
        #'early_stopping_rounds' : [30],
        'tree_method' : ['gpu_hist'],#['gpu_hist'], 
        'gpu_id' : [0],

        
}


#gpu_params = {'updater':'', 'predictor':'cpu_predictor', 'tree_method':'hist'}
result = evaluation.evaluate_model_skopt(df_training[['era']+features], df_training[target],
                                          MyXGBRanker(), 'xgb_ranker',
                                          xgb_param_grid, 
                                          scorer, 
                                          n_jobs=-1, 
                                          n_iter=1, 
                                          cv_folds=ts_grp, 
                                          pipeline=None)

#result[0].model.get_booster().set_param(gpu_params)
results['xgb_ranker'] = result

#step 1: 0.0352
#step 2: 0.0460
#step 3: 0.0460
#step 4: 0.0457
#step 5: 0.0465
#step 6: 0.0480

#### Salvando os modelos (vanilla)

In [ ]:
model_name = [
              #"lr"
              #"ridge"
              #"omp_exp20"
              #"xgb_exp20"
              "lgbm_exp20"
              #"xgb_ranker"             
]

In [ ]:
results

{'xgb_slider20': (<train_model.TunedModel at 0x7f43fd1cd6a0>,
  'xgb_slider20',
  0.7624803868757556,
  0.38353656314999585)}

In [ ]:
from joblib import dump, load

#Maior media, senao sera ultimo feito, 
#results = sorted(results, key=lambda x: x[2], reverse=True)


for name in model_name:
  #model = list(filter(lambda x: x[1] == name, results))[-1][0]
  dump(results[name][0], name + '-cv.pkl')

## 6. FN Transforms (after)

In [ ]:
%%time
path = '/content/dissertacao/reports/predicoes_validacao/raw/'
models_nr = ['ex_preds', 'ex_FN100', 'nr__rio', 'nr__sao_paulo', 'nr__medellin']


preds_nr, feat_corrs_nr = dict(), dict()
era_scores_nr, df_metrics_nr = dict(), dict()


for model in models_nr[0:]:
    
    #predicoes val1 & val2
    print("creating predictions to:", model)
    preds_nr[model]=  pd.read_csv(path+model+'_preds_test.csv', index_col='id').values.reshape(1,-1)[0]

    

    #salvando as metricas
    era_scores_nr[model], df_metrics_nr[model], feat_corrs_nr[model], ex_preds = \
                        metrics.submission_metrics(df_validation, preds_nr[model], model, True)   





#dict to dataframe
df_preds_nr = pd.DataFrame.from_dict(preds_nr)
df_era_scores_nr = pd.DataFrame.from_dict(era_scores_nr)
df_feat_corrs_nr = pd.DataFrame.from_dict(feat_corrs_nr)
df_metrics_cons_nr = metrics.metrics_consolidated(df_metrics_nr)

In [ ]:
#print("Menor é melhor:", min_cols)
leaderboard_nr = df_metrics_cons_nr[df_metrics_cons_nr.Categoria.isin(["Performance", "Risk", "MMC"])].loc[:,models_nr[0:]]
leaderboard_nr.astype(float).style.apply(visualize.diagnostic_colors).apply(visualize.highlight_max, axis = 1)

,ex_preds,ex_FN100,nr__rio,nr__sao_paulo,nr__medellin
Metrica,,,,,
Validation_Sharpe,0.9757,0.9757,0.9500,0.8757,0.9678
Validation_Mean,0.0266,0.0266,0.0264,0.0245,0.0256
Feat_neutral_mean,0.0215,0.0215,0.0206,0.0183,0.0187
Validation_SD,0.0272,0.0272,0.0278,0.0280,0.0264
Feat_exp_max,0.2694,0.2694,0.2609,0.2933,0.2653
Max_Drawdown,-0.0651,-0.0651,-0.0867,-0.0859,-0.0703
corr_plus_mmc_sharpe,0.9757,0.9757,0.8973,0.7949,0.9121
val_mmc_mean,0.0000,0.0000,0.0017,-0.0008,0.0014
corr_with_example_preds,1.0000,1.0000,0.9088,0.9631,0.8942


### MO model

Here is a slightly different take on feature neutralization. Instead of finding a linear model of your predictions and subtracting a proportion of it off, we could instead find a linear model that when subtracted off **reduces your feature exposure below a certain target.**

We could **set a target and define a loss function** such that when minimized all exposures will be less than or equal to the minimum of current exposure and the maximum desired exposure.

 So if some features have an exposure of 0.05, and you set a max exposure of 0.10, the features with the exposure of 0.05 won’t necessarily decrease as they would in the current neutralization code. 

This allows you to keep some of the smaller exposures that might be important, while reducing your largest risks. Test it out and let me know what you think! Be warned, it’s not especially fast…

In [ ]:
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F



def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)



def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)

    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])

    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):#100000
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))

        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        #print(loss)
        print(f'       loss: {loss:0.7f}', end='\r')

        if loss < 1e-7: #7
            #print('11111')
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer



def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.05):
  
    #print(max_exp)
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r') #print era
        df_era = df[df[era_col] == u]
        scores = df_era[column].values #preds
        exposure_values = df_era[neutralizers].values #features
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)


def neutralize_by_threshold(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.05):
  
  
  data_rfe = reduce_all_exposures(df, column, neutralizers, 
                                  normalize, gaussianize, 
                                  era_col="era", 
                                  max_exp=0.05)
  
  df[column] = data_rfe[column]
  df[column]  -= df[column] .min()
  df[column]  /= df[column] .max()

  return df[column]



In [ ]:
%%time

path = '/content/dissertacao/reports/predicoes_validacao/raw/'
models_nr = ['ex_preds', 'nr__rio', 'nr__sao_paulo', 'nr__medellin']

preds_nr, feat_corrs_nr = dict(), dict()
era_scores_nr, df_metrics_nr = dict(), dict()

data = df_validation.copy()
#data = data[data.era < 123]

for model in models_nr[0:]:
    
    #predicoes val1 & val2
    print("creating predictions to:", model)
    data['preds'] =  pd.read_csv(path+model+'_preds_test.csv', index_col='id').values.reshape(1,-1)[0]#[:9231]


    preds_nr[model] = neutralize_by_threshold(data, 
                                              ['preds'],
                                              features,
                                              True, True, 
                                              era_col="era",
                                              max_exp=0.06).values.reshape(1,-1)[0]
      
    
    #salvando as metricas
    era_scores_nr[model], df_metrics_nr[model], feat_corrs_nr[model], ex_preds = \
                        metrics.submission_metrics(data, preds_nr[model], model, True)   



#add ex_preds
#print("creating predictions to: ex_preds")
#preds_nr['ex_preds'] = np.array(ex_preds)
#era_scores_nr['ex_preds'], df_metrics_nr['ex_preds'], feat_corrs_nr['ex_preds'], ex_preds = \
#                        metrics.submission_metrics(data, preds_nr['ex_preds'], "ex_preds", False)



#dict to dataframe
df_preds_nr = pd.DataFrame.from_dict(preds_nr)
df_era_scores_nr = pd.DataFrame.from_dict(era_scores_nr)
df_feat_corrs_nr = pd.DataFrame.from_dict(feat_corrs_nr)
df_metrics_cons_nr = metrics.metrics_consolidated(df_metrics_nr)

In [ ]:
#print("Menor é melhor:", min_cols)
leaderboard_nr = df_metrics_cons_nr[df_metrics_cons_nr.Categoria.isin(["Performance", "Risk", "MMC",])].loc[:,models_nr[0:]]
leaderboard_nr.astype(float).style.apply(visualize.diagnostic_colors).apply(visualize.highlight_max, axis = 1)

,ex_preds,nr__rio,nr__sao_paulo,nr__medellin
Metrica,,,,
Validation_Sharpe,1.2067,1.1966,1.1203,1.2504
Validation_Mean,0.0253,0.0248,0.0228,0.0232
Feat_neutral_mean,0.0214,0.0209,0.0186,0.0193
Validation_SD,0.0209,0.0207,0.0203,0.0186
Feat_exp_max,0.0500,0.0500,0.0500,0.0500
Max_Drawdown,-0.0464,-0.0498,-0.0527,-0.0324
corr_plus_mmc_sharpe,1.2067,1.0507,0.9197,1.0624
val_mmc_mean,0.0000,0.0054,0.0030,0.0045
corr_with_example_preds,1.0000,0.6587,0.7026,0.6448


In [ ]:
#print("Menor é melhor:", min_cols)
leaderboard_nr = df_metrics_cons_nr[df_metrics_cons_nr.Categoria.isin(["Performance", "Risk", "MMC",])].loc[:,models_nr[:]]
leaderboard_nr.astype(float).style.apply(visualize.diagnostic_colors).apply(visualize.highlight_max, axis = 1)

,ex_preds,nr__rio,nr__sao_paulo,nr__medellin
Metrica,,,,
Validation_Sharpe,1.1834,1.1354,1.0361,1.1509
Validation_Mean,0.0263,0.0253,0.0230,0.0240
Feat_neutral_mean,0.0213,0.0208,0.0184,0.0192
Validation_SD,0.0222,0.0223,0.0222,0.0209
Feat_exp_max,0.1000,0.1000,0.1000,0.1000
Max_Drawdown,-0.0444,-0.0461,-0.0522,-0.0332
corr_plus_mmc_sharpe,1.1834,1.0465,0.8730,1.0609
val_mmc_mean,0.0000,0.0034,0.0006,0.0028
corr_with_example_preds,1.0000,0.7748,0.8248,0.7572


creating csv preds MO

In [ ]:
%%time

from joblib import dump, load
path =''# '../../reports/predicoes_validacao/'

models, res_cv, preds = dict(), dict(), dict()
#models_nr = ['ex_preds', 'ex_FN100', 'nr__rio', 'nr__sao_paulo', 'nr__medellin']


for model in models_nr:
    #load model complete pipe
    print("creating predictions to:", model)
    #models[model] = load(file_path + model + '-cv.pkl')
    #preds[model] = models[model].model.predict(df_validation[features])

    # predictions must have an `id` column and a `prediction_kazutsugi` column
    predictions_df = data["id"].to_frame()
    predictions_df[model] = preds_nr[model]
    predictions_df.to_csv(path+model +"_preds_test.csv", index=False)


creating predictions to: ex_preds
creating predictions to: nr__rio
creating predictions to: nr__sao_paulo
creating predictions to: nr__medellin
CPU times: user 1.06 s, sys: 16 ms, total: 1.08 s
Wall time: 1.07 s


## 7. Meta Model

### Creating Data

Get model files

In [ ]:
#import glob
#from joblib import dump, load
#file_path = '/content/dissertacao/models/sao_paulo/'

#preds = glob.glob(file_path+'*-cv.pkl')
#preds

In [ ]:
%%time
from joblib import dump, load


file_path = '/content/dissertacao/models/sao_paulo/models_gpu/'
models_meta, preds_meta = dict(), dict()


model_cv = ['lr', 'ridge', 'omp', 'xgb', 'lgbm', 'xgb_ranker']
model_ts = ['lr_ts', 'ridge_ts', 'omp_ts', 'xgb_ts', 'lgbm_ts', 'xgb_ranker_ts']
model_forest = ['xgb_forest', "lgbm_forest", "xgb_ranker_forest"]


model_names_meta = model_cv + model_ts + model_forest
#model_names_meta = ['omp', 'lr']#, 'xgb_ranker_forest']


for model in model_names_meta:
    
    #load model complete pipe
    #print("loading model:", model)
    models_meta[model] = load(file_path + model + '-cv.pkl')

#stack_data = meta_model.create_preds_meta_full(models_meta, df_training, splits=3)
#stack_data = meta_model.create_preds_meta_light(models_meta, df_training, splits=3)


#import pickle
#with open('stacked_data.pickle', 'wb') as handle: #write
#    pickle.dump(stack_data, handle, protocol=pickle.HIGHEST_PROTOCOL)



---



---



### Load Data

Get L1-Data

In [ ]:
%%time
import meta_model
from joblib import dump, load

file_path = '/content/dissertacao/models/sao_paulo/models_gpu/'
models_meta, preds_meta = dict(), dict()


model_cv = ['lr', 'ridge', 'omp', 'xgb', 'lgbm', 'xgb_ranker']
model_ts = ['lr_ts', 'ridge_ts', 'omp_ts', 'xgb_ts', 'lgbm_ts', 'xgb_ranker_ts']
model_forest = ['xgb_forest', "lgbm_forest", "xgb_ranker_forest"]
model_names_meta = model_cv + model_ts + model_forest

#get data
df_train_l1, l1_features = meta_model.get_stacked_data(meta_model="Sao_Paulo", local="colab")
#df_train_l1, l1_features = \
#              meta_model.mount_stacked_data_light(stacked_data, model_names_meta, df_training)



CPU times: user 1.72 s, sys: 73.7 ms, total: 1.8 s
Wall time: 3.02 s




---



---



### Meta Models

#### Ensemble GRP-Kfold

In [ ]:
#Group K-fold
CV = GroupKFold(n_splits = 6)
cv_grp = list(CV.split(X = df_training[features], y = df_training[target],  groups = df_training.era.values))

#Validacao usada
grp_type="group_cv" #group_cv



Metricas

In [ ]:
def spearman(target, pred):
    from scipy import stats
    return stats.spearmanr(target, pred)[0]

scorer = make_scorer(spearman)

Save Results

In [ ]:
results_meta = dict()



---



---



Ridge

In [ ]:
from sklearn.linear_model import Ridge

ridge_param_grid =  {
    'fit_intercept' : [True, False],
    'normalize' : [True, False],
    'alpha' : Real(10**-1, 10**1, "log-uniform")

}


result = evaluation.evaluate_model_skopt(df_train_l1[l1_features], 
                                         df_train_l1['target_nomi'],
                                         Ridge(), 'ridge_l1',
                                         ridge_param_grid, 
                                         scorer, 
                                         n_iter=10, 
                                         cv_folds=cv_grp, 
                                         pipeline=None)

results_meta['ridge_l1'] = result

In [ ]:
pd.DataFrame(result[0].model.coef_, index=l1_features,columns=["coef_l1"]).sort_values('coef_l1', ascending=False)

In [ ]:
from joblib import dump, load
dump(results_meta['ridge_l1'][0], 'ridge_l1' + '-cv.pkl')



---



---



LGBM

In [ ]:
from lightgbm import LGBMRegressor
#LGBMRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, num_leaves=32)

lgbm_param_grid = {
    

    #step 1 & 6
    'n_estimators' : [3870],#Integer(1500, 5000, 'uniform'), 
    'learning_rate' : [0.0011020213071919218], #Real(10**-3, 10**-1, "log-uniform"),


    #step 2
    'num_leaves': [249], #Integer(50, 250, "uniform"), #
    'min_data_in_leaf' :[1948],# Integer(100, 2000, 'uniform'), #
    'max_depth' : [3],#Integer(3, 30, 'uniform'), #
    

    #step 3
    'bagging_fraction' : [0.1384607156999449],#Real(1e-1, 1, 'uniform'), #
    'bagging_temperature': [0.22109455109869675],# Real(1e-1, 1.0, 'uniform'), #
    
    #step 4 l1 no usar
    #'colsample_bytree': Real(1e-1, .5, 'uniform'), # l1 nao usar
    #'subsample': [1] #Real(1e-1, 1.0, 'uniform'), # l1 nao usar

    #step 5
    'reg_lambda': [0.19530125277968313], #Real(10**-2, 1, "log-uniform"), #


    #utils
    'boosting_type': ['goss'],
    'device_type' : ['gpu']

}


print(grp_type)
#gpu_params = {'device_type':'cpu'}
result = evaluation.evaluate_model_skopt(df_train_l1[l1_features], 
                                         df_train_l1['target_nomi'],
                                          LGBMRegressor(), 'lgbm_l1',
                                          lgbm_param_grid, 
                                          scorer, 
                                          n_iter=1, 
                                          cv_folds=cv_grp, 
                                          pipeline=None)


#result[0].model.set_params(**gpu_params)
results_meta['lgbm_l1'] = result


#score step 1: 0.0xx
#score step 2: 0.0365
#score step 3: 0.0373
#score step 4: 0.0xx
#score step 5: 0.0369
#score step 6: 0.0478

In [ ]:
from joblib import dump, load
dump(results_meta['lgbm_l1'][0], 'lgbm_l1' + '-cv.pkl')



---



---



#### TS-GRP fold validation

In [ ]:
#Group TS-fold
ts_grp = group_ts_split.TimeSeriesSplitGroups(6, df_training.era)

#Validacao usada
grp_type="time_series" 
print(ts_grp)

TimeSeriesSplitGroups(grp=None, n_splits=6)



Metricas

In [ ]:
def spearman(target, pred):
    from scipy import stats
    return stats.spearmanr(target, pred)[0]

scorer = make_scorer(spearman)

Save Results

In [ ]:
results_meta = dict()

In [ ]:
from xgboost import XGBRegressor
#example scripts
#XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1)

xgb_param_grid = {
        #step 1 & 6
        'n_estimators' : [3010],#Integer(1000, 5000, 'uniform'),
        'learning_rate' : [0.021470082387096502],#Real(10**-3, 10**-1, "log-uniform"),


        #step 2
        'max_depth' : [3],#Integer(3, 10, 'uniform'),
        'min_child_weight':[1],# Integer(1, 10, 'uniform'),

        #step 3
        'gamma' : [0.08626899377510583],#Real(1e-5, 6, 'log-uniform'),


        #step 4
        'subsample' : [0.9983396500245366],#Real(1e-1, 1, 'uniform'),
        'colsample_bytree':[0.49725039069087584], #Real(1e-1, .5, 'uniform'), 

        #step 5
        'reg_alpha' : [2.794697948644621],#Real(1, 100, 'log-uniform'),


        #utils
        'objective': ["reg:squarederror"],
        #'early_stopping_rounds' : [30],
        'tree_method' : ['hist'],#['gpu_hist'], 
        #'gpu_id' : [0],
        
}


#gpu_params = {'updater':'', 'predictor':'cpu_predictor', 'tree_method':'hist'}
result = evaluation.evaluate_model_skopt(df_train_l1[l1_features], 
                                         df_train_l1['target_nomi'],
                                          XGBRegressor(), 'xgb_ts_l1',
                                          xgb_param_grid, 
                                          scorer, 
                                          n_jobs=-1, 
                                          n_iter=1, 
                                          cv_folds=ts_grp, 
                                          pipeline=None)


#result[0].model.get_booster().set_param(gpu_params)
results_meta['xgb_ts_l1'] = result

#score step 1: 0.0
#score step 2: 0.0
#score step 3: 0.0
#score step 4: 0.0
#score step 5: 0.0
#score step 6: 0.0401

In [ ]:
from joblib import dump, load
dump(results_meta['xgb_ts_l1'][0], 'xgb_ts_l1' + '-cv.pkl')



---



---



#### Block K-fold (forest)

In [ ]:
def spearman(target, pred):
    from scipy import stats
    return stats.spearmanr(target, pred)[0]
results_meta = dict()

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize
from lightgbm import LGBMRegressor


#####################################################################
#####################################################################

space  = [
           #step 1&5
           Integer(1000, 1010, name='n_estimators', prior='uniform'),
           #Real(5**-3, 10**-1,  name='learning_rate', prior='log-uniform'),

           #step 2
           #Integer(50, 250, name='num_leaves', prior='uniform'),
           #Integer(100, 2000, name='min_data_in_leaf', prior='uniform'),
           #Integer(3, 30, name='max_depth', prior='uniform'),

           #step 3 nao usar no l1
           #Real(5e-2, 0.5, name='colsample_bytree', prior='uniform'),
           #Real(5e-2, 0.5, name='subsample', prior='uniform'),

           #step 4
           #Real(10**-2, 1.0, name='reg_lambda', prior='log-uniform'),
]

#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    print(params)

    data['block'] = np.trunc(data['era']/20).astype(int)
    data.loc[data['block'] == 6, 'block'] = 5

    results_val=[]
    for block in range(6):
        #print("Validation Block {}".format(block)) 

        train = data[data['block'] != block]
        val = data[data['block'] == block]

        X_train = train.filter(regex=r'feature')
        X_val = val.filter(regex=r'feature')

        y_train = train['target_nomi']
        y_val = val['target_nomi']
        

        model_.train(X_train[l1_features], y_train)
        preds_val_df = pd.Series(model.predict(X_val[l1_features]), index=X_val.index)


        correlation = spearman(y_val, preds_val_df)
        results_val.append(correlation)
        print("Correlation  {}".format(correlation))

    #save best score
    model_.save_cv_scores(results_val)
    return -np.mean(results_val)


#####################################################################
#####################################################################

#gpu_params = {'device_type':'cpu'}
model= LGBMRegressor(boosting_type='goss', 
                     device_type = "gpu",
                     n_estimators=1002,
                     learning_rate=0.0085646,
                     num_leaves=250,
                     min_data_in_leaf=2000,
                     max_depth=3,
                     colsample_bytree = 1, #l1
                     subsample = 1, #l1
                     reg_lambda=0.30464128088558723,
                     n_jobs=-1)


param = model.get_params()
model_ = train_model.Model('lgbm_forest_l1', model, n_iter=1, cv_folds=1,
                            n_jobs = -1, pipeline=None, fit_params=param)
  


#####################################################################
#####################################################################

data = df_train_l1.copy()
res_gp = forest_minimize(objective, space, n_calls=1, n_random_starts=1, verbose=1, random_state=123)
#model.set_params(**gpu_params)
results_meta['lgbm_forest_l1'] = model_

##Lembre de salvar os cv scores certos
#model_.save_cv_scores(results_val)


#score step 1: 0.0xx
#score step 2: 0.0350
#score step 3: 0.0xx
#score step 4: 0.0xx
#score step 5: 0.0401
#score step 6: 0.0480

In [ ]:
model_.results

In [ ]:
print("fez os 30 com sucesso")
print(res_gp.func_vals)
print(res_gp.x_iters)
print(res_gp.x)

In [ ]:
from joblib import dump, load
dump(results_meta['lgbm_forest_l1'], 'lgbm_forest_l1' + '-cv.pkl')

['lgbm_forest_l1-cv.pkl']



---



---



#### Loop aninhado

In [ ]:
#get data
stacked_data = meta_model.get_stacked_data_full(meta_model="Sao_Paulo", local="colab")

def spearman(target, pred):
    from scipy import stats
    return stats.spearmanr(target, pred)[0]

    
results_meta = dict()

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize
from xgboost import XGBRegressor


#####################################################################
#####################################################################


space  = [
           #step 1 & 6
           Integer(500, 3000, name='n_estimators', prior='uniform'), 
           #Real(5**-3, 10**-1, name='learning_rate', prior='log-uniform'), 

           #step 2
           #Integer(3, 10, name='max_depth', prior='uniform'),
           #Integer(1, 10, name='min_child_weight', prior='uniform'),

           #step 3
           #Real(1e-5, 6, name='gamma', prior='log-uniform'),

           #step 4 l1 nao usar
           #Real(5e-2, .5, name='colsample_bytree', prior='uniform'),
           #Real(5e-2, 1.0, name='subsample', prior='uniform'),
          
           #step 5
           #Real(1, 100, name='reg_alpha', prior='log-uniform'),

]

#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    print(params)

    results_val=[]
    for fold in data.keys():
        #print("Training on {}".format(fold))

        X_train = data[fold]['Xtrain']
        X_val = data[fold]['Xtest']

        y_train = data[fold]['ytrain']
        y_val = data[fold]['ytest']


        model_.train(X_train, y_train)
        correlation = spearman(y_val, model.predict(X_val))
        results_val.append(correlation)

        print("Correlation  {}".format(correlation))
        #print()

    #save best score
    model_.save_cv_scores(results_val)
    return -np.mean(results_val)


#####################################################################
#####################################################################

#gpu_params = {'updater':'', 'predictor':'cpu_predictor', 'tree_method':'hist'}
model =XGBRegressor(tree_method = "hist",
                    learning_rate = 0.0515946341276062,
                    n_estimators=4280,
                    max_depth=3,
                    min_child_weight=5,
                    gamma =0.13169329887735526,
                    colsample_bytree=1,
                    subsample=1,
                    reg_alpha=7.193526575307788,
                    n_jobs=-1)
                    #gpu_id=0)


param = model.get_params()
model_ = train_model.Model('xgb_loop_l1', model, n_jobs=-1, cv_folds=1, 
                           n_iter=1, pipeline=None, fit_params=param)

#####################################################################
#####################################################################
data = stacked_data.copy()
res_gp = forest_minimize(objective, space, n_calls=1, n_random_starts=1, verbose=1, random_state=123)
#model.set_params(**gpu_params)
results_meta['xgb_loop_l1'] = model_

##Lembre de salvar os cv scores certos
#model_.save_cv_scores(results_val)

#step 1: 0.0475

In [ ]:
print("fez os 30 com sucesso")
print(res_gp.func_vals)
print(res_gp.x_iters)
print(res_gp.x)

In [ ]:
from joblib import dump, load
dump(results_meta['xgb_loop_l1'], 'xgb_loop_l1' + '-cv.pkl')



---



---



## 8. Verificando Modelos

In [ ]:
import glob
#from joblib import dump, load
#file_path = '/content/dissertacao/models/sao_paulo/'
file_path = '/content/dissertacao/models/sao_paulo/models_gpu/'

files_path_list = glob.glob(file_path+'*-cv.pkl')



for file_ in files_path_list:
    
    #load model complete pipe
    #print("creating predictions to:", model)
    models_meta[file_] = load(file_)#(file_path + model + '-cv.pkl') #-120

In [ ]:
models_meta

{'/content/dissertacao/models/sao_paulo/models_gpu/lgbm-cv.pkl': <train_model.TunedModel_Skopt at 0x7f866793c978>,
 '/content/dissertacao/models/sao_paulo/models_gpu/lgbm_forest-cv.pkl': <train_model.Model at 0x7f8619b6b1d0>,
 '/content/dissertacao/models/sao_paulo/models_gpu/lgbm_forest_l1-cv.pkl': <train_model.Model at 0x7f8619bc1828>,
 '/content/dissertacao/models/sao_paulo/models_gpu/lgbm_l1-cv.pkl': <train_model.TunedModel_Skopt at 0x7f8619bf7ef0>,
 '/content/dissertacao/models/sao_paulo/models_gpu/lgbm_ts-cv.pkl': <train_model.TunedModel_Skopt at 0x7f8619c0dac8>,
 '/content/dissertacao/models/sao_paulo/models_gpu/lr-cv.pkl': <train_model.TunedModel at 0x7f8619a73cc0>,
 '/content/dissertacao/models/sao_paulo/models_gpu/lr_ts-cv.pkl': <train_model.TunedModel at 0x7f8619bf7710>,
 '/content/dissertacao/models/sao_paulo/models_gpu/omp-cv.pkl': <train_model.TunedModel_Skopt at 0x7f8619c1e550>,
 '/content/dissertacao/models/sao_paulo/models_gpu/omp_ts-cv.pkl': <train_model.TunedModel_Sk

In [ ]:
len(files_path_list)

20

## 9. Special Models

Split Data 1/3 eras

In [ ]:
eras_val = df_training.era.unique()[::3]

df_special_train = df_training[~df_training.era.isin(eras_val)]
df_special_val = df_training[df_training.era.isin(eras_val)]

eras_val_ind = df_special_val.era

Metrica

In [ ]:
def numerai_score(y_true, y_pred):
    rank_pred = y_pred.groupby(eras_val_ind).apply(lambda x: x.rank(pct=True, method="first"))
    return np.corrcoef(y_true, rank_pred)[0,1]


Save Results

In [ ]:
special_models = dict()

#### Xgboost_Sharpe_Forest

In [ ]:

target = "target_nomi" 
path = "/content/dissertacao/models/sao_paulo/"
name = "lgbm"

# fit an initial model "lgbm_vanilla"
model_base = load(path + name + '-cv.pkl') #deveria ser 120


#model_base.fit(df_training[features], df_training[target]) # se necessário
base_margin = model_base.model.predict(df_special_train[features])

In [ ]:
import torch
from torch.autograd import grad


# get indexes for each era
era_idx = [np.where(df_special_train.era==uera)[0] for uera in df_special_train.era.unique()]

# use correlation as the measure of fit
def corr(pred, target):
    pred_n = pred - pred.mean(dim=0)
    pred_n = pred_n / pred_n.norm(dim=0)

    target_n = target - target.mean(dim=0)
    target_n = target_n / target_n.norm(dim=0)
    l = torch.matmul(pred_n.T, target_n)
    return l


# definte a custom objective for XGBoost
def adj_sharpe_obj(ytrue, ypred):
    # convert to pytorch tensors
    ypred_th = torch.tensor(ypred, requires_grad=True)
    ytrue_th = torch.tensor(ytrue)
    all_corrs = []

    # get correlations in each era
    for ee in era_idx:
        score = corr(ypred_th[ee], ytrue_th[ee])
        all_corrs.append(score)

    all_corrs = torch.stack(all_corrs)

    # calculate adjusted sharpe using correlations
    loss = -metrics.adj_sharpe(all_corrs)
    print(f'Current loss:{loss}')

    # calculate gradient and convert to numpy
    loss_grads = grad(loss, ypred_th, create_graph=True)[0]
    loss_grads = loss_grads.detach().numpy()

    # return gradient and ones instead of Hessian diagonal
    return loss_grads, np.ones(loss_grads.shape)


#model_adj_sharpe = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=200, nthread=6, colsample_bytree=0.1, objective=adj_sharpe_obj)
#model_adj_sharpe.fit(df_training[feature_columns], df_training[target], base_margin=base_margin)

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize
from xgboost import XGBRegressor

#####################################################################
#####################################################################

space  = [
           #step 1 & 6
           Integer(10, 12, name='n_estimators', prior='uniform'), 
           #Real(5**-3, 10**-1, name='learning_rate', prior='log-uniform'), 

           #step 2
           #Integer(3, 30, name='max_depth', prior='uniform'),
           #Integer(1, 30, name='min_child_weight', prior='uniform'),

           #step 3
           #Real(1e-5, 6, name='gamma', prior='log-uniform'),

           #step 4
           #Real(5e-2, 1.0, name='colsample_bytree', prior='uniform'),
           #Real(5e-2, 1.0, name='subsample', prior='uniform'),


]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    print(params)


    model.fit(df_special_train[features], df_special_train[target],  base_margin=base_margin)
    preds_df = pd.Series(model.predict(df_special_val[features]), index=df_special_val.index)
    

    return -numerai_score(df_special_val[target], preds_df)


#####################################################################
#####################################################################


gpu_params = {'updater':'', 'predictor':'cpu_predictor', 'tree_method':'hist'}
model = XGBRegressor(tree_method = "gpu_hist",
                     objective=adj_sharpe_obj,
                     learning_rate = 0.1,
                     #n_estimators= 727,
                     #max_depth=4,
                     #min_child_weight=7,
                     #gamma = 5.7794,
                     #colsample_bytree=0.2258,
                     #subsample=0.9349,
                     #reg_alpha=72.96,
                     n_jobs=-1,
                     gpu_id=0)


#####################################################################
#####################################################################



special_models['xgb_sharpe'] = model
res_gp = forest_minimize(objective, space, n_calls=1, n_random_starts=1, verbose=1, random_state=6)
model.get_booster().set_param(gpu_params)

In [ ]:
print("fez os 30 com sucesso")
print(res_gp.func_vals)
print(res_gp.x_iters)
print(res_gp.x)



---



---



#### Era boosted

In [ ]:
def ar1(x):
    return np.corrcoef(x[:-1], x[1:])[0,1]

def autocorr_penalty(x):
    n = len(x)
    p = ar1(x)
    return np.sqrt(1 + 2*np.sum([((n - i)/n)*p**i for i in range(1,n)]))

def smart_sharpe(x):
    return np.mean(x)/(np.std(x, ddof=1)*autocorr_penalty(x))

import matplotlib.pyplot as plt
def era_boost_train(X, y, era_col, proportion=0.5, trees_per_step=10, num_iters=200):
    
    
    model = GradientBoostingRegressor(max_depth=5, learning_rate=0.01, 
                                      max_features="sqrt", subsample=0.5, 
                                      n_estimators=trees_per_step, 
                                      warm_start=(num_iters>1))
    
    features = X.columns
    model.fit(X, y)
    new_df = X.copy()
    new_df["target"] = y
    new_df["era"] = era_col
    
    
    for i in range(num_iters-1):
        print(f"iteration {i}")
        # score each era
        print("predicting on train")
        preds = model.predict(X)
        new_df["pred"] = preds
        era_scores = pd.Series(index=new_df["era"].unique())
        
        print("getting per era scores")
        for era in new_df["era"].unique():
            era_df = new_df[new_df["era"] == era]
            era_scores[era] = spearmanr(era_df["pred"], era_df["target"])[0]
            
            
        era_scores.sort_values(inplace=True)
        worst_eras = era_scores[era_scores <= era_scores.quantile(proportion)].index
        print(list(worst_eras))
        worst_df = new_df[new_df["era"].isin(worst_eras)]
        era_scores.sort_index(inplace=True)
        era_scores.plot(kind="bar")
        print("performance over time")
        plt.show()
        print("autocorrelation")
        print(ar1(era_scores))
        print("mean correlation")
        print(np.mean(era_scores))
        print("sharpe")
        print(np.mean(era_scores)/np.std(era_scores))
        print("smart sharpe")
        print(smart_sharpe(era_scores))
        model.n_estimators += trees_per_step
        print("fitting on worst eras")
        model.fit(worst_df[features], worst_df["target"])
    return model


boost_model = era_boost_train(train_features, train_targets["target_kazutsugi"], 
                              era_col=train_targets["era"], proportion=0.5, 
                              trees_per_step=10, num_iters=20)

In [ ]:
from xgboost import XGBRegressor

def spearmanr(target, pred):
    return np.corrcoef(
        target,
        pred.rank(pct=True, method="first")
    )[0, 1]

def era_boost_train(X, y, era_col, proportion=0.5, trees_per_step=10, num_iters=200):
    
    
    model = XGBRegressor(max_depth=5, learning_rate=0.01, 
                         n_estimators=trees_per_step, 
                         n_jobs=-1, colsample_bytree=0.1)
    
    
    features = X.columns
    model.fit(X, y)
    new_df = X.copy()
    new_df["target"] = y
    new_df["era"] = era_col
    for i in range(num_iters-1):
        print(f"iteration {i}")
        # score each era
        print("predicting on train")
        preds = model.predict(X)
        new_df["pred"] = preds
        era_scores = pd.Series(index=new_df["era"].unique())
        print("getting per era scores")
        for era in new_df["era"].unique():
            era_df = new_df[new_df["era"] == era]
            era_scores[era] = spearmanr(era_df["pred"], era_df["target"])
        era_scores.sort_values(inplace=True)
        worst_eras = era_scores[era_scores <= era_scores.quantile(proportion)].index
        print(list(worst_eras))
        worst_df = new_df[new_df["era"].isin(worst_eras)]
        era_scores.sort_index(inplace=True)
        era_scores.plot(kind="bar")
        print("performance over time")
        plt.show()
        print("autocorrelation")
        print(ar1(era_scores))
        print("mean correlation")
        print(np.mean(era_scores))
        print("sharpe")
        print(np.mean(era_scores)/np.std(era_scores))
        print("smart sharpe")
        print(smart_sharpe(era_scores))
        model.n_estimators += trees_per_step
        booster = model.get_booster()
        print("fitting on worst eras")
        model.fit(worst_df[features], worst_df["target"], xgb_model=booster)
    return model

boost_model = era_boost_train(train_features, train_targets["target_kazutsugi"], 
                              era_col=train_targets["era"], proportion=0.5, 
                              trees_per_step=10, num_iters=20)

#### Ranker_Forest

In [ ]:
#group 1/3 dataset
cdf = df_special_train.groupby('era').agg(['count'])
group_special = cdf[cdf.columns[0]].values

In [ ]:
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize
from xgboost import XGBRanker

#####################################################################
#####################################################################

space  = [
           #step 1 & 6
           #Integer(2000, 5000, name='n_estimators', prior='uniform'), 
           #Real(5**-3, 10**-1, name='learning_rate', prior='log-uniform'), 

           #step 2
           Integer(3, 10, name='max_depth', prior='uniform'),
           Integer(1, 10, name='min_child_weight', prior='uniform'),

           #step 3
           #Real(1e-5, 6, name='gamma', prior='log-uniform'),

           #step 4
           Real(5e-2, 1.0, name='colsample_bytree', prior='uniform'),
           Real(5e-2, 1.0, name='subsample', prior='uniform'),
          
          #step 5
          #Real(1, 100, name='reg_alpha', prior='log-uniform'),

]


#####################################################################
#####################################################################

@use_named_args(space)
def objective(**params):
    model.set_params(**params)
    print(params)

    model.fit(df_special_train[features], df_special_train[target], group=group_special)
    preds_df = pd.Series(model.predict(df_special_val[features]), index=df_special_val.index)
    
    return -numerai_score(df_special_val[target], preds_df)

#####################################################################
#####################################################################


gpu_params = {'updater':'', 'predictor':'cpu_predictor', 'tree_method':'hist'}
model = XGBRanker(tree_method = "gpu_hist",
                  learning_rate = 0.038900611770001495,
                  n_estimators= 4731,
                  max_depth=4,
                  min_child_weight=8,
                  gamma = 0.3285961066329761,
                  colsample_bytree=0.13505449410675752,
                  subsample=0.6468152889357408,
                  reg_alpha=47.56020058180222,
                  n_jobs=-1,
                  gpu_id=0)


#####################################################################
#####################################################################


res_gp = forest_minimize(objective, space, n_calls=30, n_random_starts=10, verbose=1, random_state=23)
model.get_booster().set_param(gpu_params)
forest_models['xgb_ranker'] = model



#step 1: 0.0289
#step 2: 0.0371
#step 3: 0.0371
#step 4: 0.0371
#step 5: 0.0371
#step 6: 0.0383/0.0391/0.0

In [ ]:
print("fez os 30 com sucesso")
print(res_gp.func_vals)
print(res_gp.x_iters)
print(res_gp.x)



---



---

